# Inference

## An inferential problem: Wage Gap based on Sex

In the previous lab, we already analyzed data from the March Supplement of the U.S. Current Population Survey (2015) and answered the question how to use job-relevant characteristics, such as education and experience, to best predict wages. Now, we focus on the following inference question:


**What is the difference in predicted wages between male and female workers with the same job-relevant characteristics?**

Thus, we analyze if there is a difference in the payment of male and female workers (wage gap). The wage gap may partly reflect discrimination against female workers in the labor market or may partly reflect a selection effect, namely that female workers are relatively more likely to take on occupations that pay somewhat less (for example, school teaching).

To investigate the wage gap, we consider the following log-linear regression model

\begin{align} \log(Y) &= \beta'X + \epsilon\\ &= \beta_1 D + \beta_2' W + \epsilon, \end{align}
where $D$ is the indicator of being female ($1$ if female and $0$ otherwise) and the $W$'s are controls explaining variation in wages. Considering transformed wages by the logarithm, we are analyzing the relative difference in the payment of male and female workers.

In [91]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import sklearn.linear_model as lm
import statsmodels.formula.api as smf
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import warnings
# ignore potential convergence warnings; for some small penalty levels, tried out, optimization might not converge
warnings.simplefilter('ignore')

## Data Analysis

We consider the same subsample of the U.S. Current Population Survey (2015) as in the previous lab. Let us load the data set.

In [92]:
file = "https://raw.githubusercontent.com/CausalAIBook/MetricsMLNotebooks/main/data/wage2015_subsample_inference.csv"
df = pd.read_csv(file)

To start our analysis, we compare the sample means for the different sexes:

In [93]:
df.head()

,wage,lwage,sex,shs,hsg,scl,clg,ad,mw,so,we,ne,exp1,exp2,exp3,exp4,occ,occ2,ind,ind2
0,9.615385,2.263364,1,0,0,0,1,0,0,0,0,1,7.0,0.49,0.343,0.2401,3600.0,11,8370.0,18
1,48.076923,3.872802,0,0,0,0,1,0,0,0,0,1,31.0,9.61,29.791,92.3521,3050.0,10,5070.0,9
2,11.057692,2.403126,0,0,1,0,0,0,0,0,0,1,18.0,3.24,5.832,10.4976,6260.0,19,770.0,4
3,13.942308,2.634928,1,0,0,0,0,1,0,0,0,1,25.0,6.25,15.625,39.0625,420.0,1,6990.0,12
4,28.846154,3.361977,1,0,0,0,1,0,0,0,0,1,22.0,4.84,10.648,23.4256,2015.0,6,9470.0,22


In [94]:
table = pd.DataFrame()

cols = ["lwage","sex","shs","hsg","scl","clg","ad","ne","mw","so","we","exp1"]

table['Variable'] = ["Log Wage","Sex","Less then High School","High School Graduate","Some College",
                      "Gollage Graduate","Advanced Degree", "Northeast","Midwest","South","West","Experience"]

table['All'] = df[cols].mean().values

table['Male'] = df[df['sex']==0][cols].mean().values
table['Female'] = df[df['sex']==1][cols].mean().values


# Show results
table

,Variable,All,Male,Female
0,Log Wage,2.970787,2.987830,2.949485
1,Sex,0.444466,0.000000,1.000000
2,Less then High School,0.023301,0.031807,0.012669
3,High School Graduate,0.243883,0.294303,0.180865
4,Some College,0.278058,0.273331,0.283967
5,Gollage Graduate,0.317670,0.293953,0.347313
6,Advanced Degree,0.137087,0.106606,0.175186
7,Northeast,0.227767,0.221950,0.235037
8,Midwest,0.259612,0.259000,0.260376
9,South,0.296505,0.298148,0.294452


In [95]:
df.describe()

,wage,lwage,sex,shs,hsg,scl,clg,ad,mw,so,we,ne,exp1,exp2,exp3,exp4,occ,occ2,ind,ind2
count,5150.000000,5150.000000,5150.000000,5150.000000,5150.000000,5150.000000,5150.000000,5150.000000,5150.000000,5150.000000,5150.000000,5150.000000,5150.000000,5150.000000,5150.000000,5150.000000,5150.000000,5150.000000,5150.000000,5150.000000
mean,23.410410,2.970787,0.444466,0.023301,0.243883,0.278058,0.317670,0.137087,0.259612,0.296505,0.216117,0.227767,13.760583,3.018925,8.235867,25.118038,5310.737476,11.670874,6629.154951,13.316893
std,21.003016,0.570385,0.496955,0.150872,0.429465,0.448086,0.465616,0.343973,0.438464,0.456761,0.411635,0.419432,10.609465,4.000904,14.488962,53.530225,11874.356080,6.966684,5333.443992,5.701019
min,3.021978,1.105912,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,1.000000,370.000000,2.000000
25%,13.461538,2.599837,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,0.250000,0.125000,0.062500,1740.000000,5.000000,4880.000000,9.000000
50%,19.230769,2.956512,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,1.000000,1.000000,1.000000,4040.000000,13.000000,7370.000000,14.000000
75%,27.777778,3.324236,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,21.000000,4.410000,9.261000,19.448100,5610.000000,17.000000,8190.000000,18.000000
max,528.845673,6.270697,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,47.000000,22.090000,103.823000,487.968100,100000.000000,22.000000,100000.000000,22.000000


In [96]:
# print to Latex
print(table.style.to_latex())

\begin{tabular}{llrrr}
 & Variable & All & Male & Female \\
0 & Log Wage & 2.970787 & 2.987830 & 2.949485 \\
1 & Sex & 0.444466 & 0.000000 & 1.000000 \\
2 & Less then High School & 0.023301 & 0.031807 & 0.012669 \\
3 & High School Graduate & 0.243883 & 0.294303 & 0.180865 \\
4 & Some College & 0.278058 & 0.273331 & 0.283967 \\
5 & Gollage Graduate & 0.317670 & 0.293953 & 0.347313 \\
6 & Advanced Degree & 0.137087 & 0.106606 & 0.175186 \\
7 & Northeast & 0.227767 & 0.221950 & 0.235037 \\
8 & Midwest & 0.259612 & 0.259000 & 0.260376 \\
9 & South & 0.296505 & 0.298148 & 0.294452 \\
10 & West & 0.216117 & 0.220902 & 0.210135 \\
11 & Experience & 13.760583 & 13.783992 & 13.731324 \\
\end{tabular}



In particular, the table above shows that the difference in average logwage between male and female workers is equal to  0,038.

In [97]:
# compute difference
table.loc[0, 'Female'] - table.loc[0, 'Male']

-0.03834473367441449

Thus, the unconditional wage gap is about  $3.8\%$ for the group of never married workers (female workers get paid less on average in our sample). We also observe that never married female workers are relatively more educated than male workers and have lower working experience.

This unconditional (predictive) effect of sex equals the coefficient  $\beta$  in the univariate ols regression of  $Y$  on  $D$ :

$$
log(Y)=\beta D+\epsilon.
$$

We verify this by running an ols regression.

In [98]:
nocontrol_fit = smf.ols("lwage ~ sex", data=df).fit()
nocontrol_est = nocontrol_fit.params['sex']
nocontrol_se = nocontrol_fit.HC3_se['sex']

print(f"The estimated sex coefficient is {nocontrol_est:.4f} "
      f"and the corresponding robust standard error is {nocontrol_se:.4f}")

The estimated sex coefficient is -0.0383 and the corresponding robust standard error is 0.0159


Note that the standard error is computed to be robust to heteroskedasticity.

Next, we run an ols regression of  $Y$  on  $(D,W)$  to control for the effect of covariates summarized in  $W$ :

$$
log(Y)=\beta_1 D + \beta_2' W + \epsilon.
$$

Here, we are considering the flexible model from the previous lab. Hence,  $W$ controls for experience, education, region, and occupation and industry indicators plus transformations and two-way interactions.

Let us run the ols regression with controls.

In [99]:
# Ols regression with controls

flex = "lwage ~ sex + (exp1+exp2+exp3+exp4)*(shs+hsg+scl+clg+C(occ2)+C(ind2)+mw+so+we)"

#   Note that ()*() operation in formula objects in statsmodels creates a formula of the sort:
#  (exp1+exp2+exp3+exp4)+ (shs+hsg+scl+clg+occ2+ind2+mw+so+we) +  (exp1+exp2+exp3+exp4)*(shs+hsg+scl+clg+occ2+ind2+mw+so+we)
#  This is not intuitive at all, but that's what it does.

control_fit = smf.ols(flex, data=df).fit()
control_est = control_fit.params['sex']
control_se = control_fit.HC3_se['sex']

control_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  lwage   R-squared:                       0.351
Model:                            OLS   Adj. R-squared:                  0.319
Method:                 Least Squares   F-statistic:                     10.83
Date:                Wed, 07 Feb 2024   Prob (F-statistic):          2.69e-305
Time:                        19:26:18   Log-Likelihood:                -3301.9
No. Observations:                5150   AIC:                             7096.
Df Residuals:                    4904   BIC:                             8706.
Df Model:                         245                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept              3.8603      0.429      9.006      0.000       3.020       4.701
C(occ2)[T.2]           0.1613      0.130      1.244      0.214      -0.093       0.416
C(occ2)[T.3]           0.2102      0.169      1.246      0.213      -0.121       0.541
C(occ2)[T.4]           0.0709      0.184      0.386      0.700      -0.289       0.431
C(occ2)[T.5]          -0.3960      0.189     -2.100      0.036      -0.766      -0.026
C(occ2)[T.6]          -0.2311      0.187     -1.236      0.217      -0.598       0.135
C(occ2)[T.7]           0.3147      0.194      1.621      0.105      -0.066       0.695
C(occ2)[T.8]          -0.1875      0.169     -1.108      0.268      -0.519       0.144
C(occ2)[T.9]          -0.3390      0.167     -2.027      0.043      -0.667      -0.011
C(occ2)[T.10]          0.0210      0.156      0.134      0.893      -0.286       0.328
C(occ2)[T.11]         -0.6424      0.309     -2.078      0.038      -1.248      -0.036
C(occ2)[T.12]         -0.0675      0.252     -0.268      0.789      -0.562       0.427
C(occ2)[T.13]         -0.2330      0.232     -1.006      0.314      -0.687       0.221
C(occ2)[T.14]          0.2562      0.323      0.794      0.427      -0.376       0.889
C(occ2)[T.15]         -0.1939      0.260     -0.747      0.455      -0.703       0.315
C(occ2)[T.16]         -0.0551      0.147     -0.375      0.708      -0.343       0.233
C(occ2)[T.17]         -0.4156      0.136     -3.053      0.002      -0.682      -0.149
C(occ2)[T.18]         -0.4822      1.044     -0.462      0.644      -2.530       1.565
C(occ2)[T.19]         -0.2579      0.333     -0.776      0.438      -0.910       0.394
C(occ2)[T.20]         -0.3010      0.234     -1.286      0.199      -0.760       0.158
C(occ2)[T.21]         -0.4272      0.221     -1.936      0.053      -0.860       0.005
C(occ2)[T.22]         -0.8695      0.298     -2.922      0.003      -1.453      -0.286
C(ind2)[T.3]          -1.2474      0.645     -1.932      0.053      -2.513       0.018
C(ind2)[T.4]          -0.0948      0.464     -0.205      0.838      -1.004       0.814
C(ind2)[T.5]          -0.5294      0.435     -1.218      0.223      -1.381       0.323
C(ind2)[T.6]          -0.6222      0.435     -1.431      0.152      -1.474       0.230
C(ind2)[T.7]          -0.5047      0.502     -1.005      0.315      -1.490       0.480
C(ind2)[T.8]          -0.7295      0.467     -1.561      0.119      -1.646       0.187
C(ind2)[T.9]          -0.8025      0.425     -1.887      0.059      -1.636       0.031
C(ind2)[T.10]         -0.5806      0.481     -1.207      0.227      -1.523       0.362
C(ind2)[T.11]         -0.9852      0.448     -2.198      0.028      -1.864      -0.107
C(ind2)[T.12]         -0.7376      0.424     -1.738      0.082      -1.569       0.094
C(ind2)[T.13]         -1.0183      0.4

In [100]:
print(f"The estimated sex coefficient is {control_est:.4f} "
      f"and the corresponding robust standard error is {control_se:.4f}")

The estimated sex coefficient is -0.0696 and the corresponding robust standard error is 0.0157


The estimated regression coefficient  $\beta_1≈-0.06955$  measures how our linear prediction of wage changes if we set the sex variable  $D$  from 0 to 1, holding the controls  $W$  fixed. We can call this the predictive effect (PE), as it measures the impact of a variable on the prediction we make. Overall, we see that the unconditional wage gap of size  4 \% for female workers increases to about  7 \% after controlling for worker characteristics.

Next, we are using the Frisch-Waugh-Lovell theorem from the lecture partialling-out the linear effect of the controls via ols.

In [101]:
# Partialling-Out using ols

# models
flex_y = "lwage ~  (exp1+exp2+exp3+exp4)*(shs+hsg+scl+clg+C(occ2)+C(ind2)+mw+so+we)" # model for Y
flex_d = "sex ~ (exp1+exp2+exp3+exp4)*(shs+hsg+scl+clg+C(occ2)+C(ind2)+mw+so+we)"   # model for D

# partialling-out the linear effect of W from Y
t_Y = smf.ols(flex_y, data=df).fit().resid
# partialling-out the linear effect of W from D
t_D = smf.ols(flex_d, data=df).fit().resid

# regression of Y on D after partialling-out the effect of W
partial_fit = sm.OLS(t_Y, t_D).fit()
partial_est = partial_fit.params['x1']

print("Coefficient for D via partialling-out " + str(partial_est))

# standard error
partial_se = partial_fit.HC3_se['x1']

# confidence interval
print("95% CI: " + str(partial_fit.conf_int().values[0]))

Coefficient for D via partialling-out -0.0695532032968458
95% CI: [-0.09866859 -0.04043781]


In [102]:
print(f"The estimated sex coefficient is {partial_est:.4f} "
      f"and the corresponding robust standard error is {partial_se:.4f}")

The estimated sex coefficient is -0.0696 and the corresponding robust standard error is 0.0150


Again, the estimated coefficient measures the linear predictive effect (PE) of  $D$  on  $Y$  after taking out the linear effect of  $W$  on both of these variables. This coefficient equals the estimated coefficient from the ols regression with controls.

We know that the partialling-out approach works well when the dimension of $W$ is low in relation to the sample size  $n$ . When the dimension of  $W$  is relatively high, we need to use variable selection or penalization for regularization purposes.


In the following, we illustrate the partialling-out approach using lasso instead of ols.

For Lasso, we use "plug-in" tuning with a theoretically valid choice of penalty $\lambda = 2 \cdot c \hat{\sigma} \sqrt{n} \Phi^{-1}(1-\alpha/2p)$, where $c>1$ and $1-\alpha$ is a confidence level, and $\Phi^{-1}$ denotes the quantile function. Under homoskedasticity, this choice ensures that the Lasso predictor is well behaved, delivering good predictive performance under approximate sparsity. In practice, this formula will work well even in the absence of homoskedasticity, especially when the random variables $\epsilon$ and $X$ in the regression equation decay quickly at the tails.

In practice, many people choose to use cross-validation, which is perfectly fine for predictive tasks. However, when conducting inference, to make our analysis valid we will require cross-fitting in addition to cross-validation. As we have not yet discussed cross-fitting, we rely on this theoretically-driven penalty in order to allow for accurate inference in the upcoming notebooks.

We pull an analogue of R's rlasso. Rlasso functionality: it is searching the right set of regressors. This function was made for the case of ***p*** regressors and ***n*** observations where ***p >>>> n***. It assumes that the error is i.i.d. The errors may be non-Gaussian or heteroscedastic.\
The post lasso function makes OLS with the selected ***T*** regressors.
To select those parameters, they use $\lambda$ as variable to penalize\
**Funny thing: the function rlasso was named like that because it is the "rigorous" Lasso.**\
We find a Python code that tries to replicate the main function of hdm r-package. It was made by [Max Huppertz](https://maxhuppertz.github.io/code/). His library is this [repository](https://github.com/maxhuppertz/hdmpy). If not using colab, download its repository and copy this folder to your site-packages folder. In my case it is located here ***C:\Python\Python38\Lib\site-packages*** . We need to install this package ***pip install multiprocess***.

In [103]:
!pip install multiprocess
!pip install pyreadr
!git clone https://github.com/maxhuppertz/hdmpy.git

fatal: destination path 'hdmpy' already exists and is not an empty directory.


In [104]:
import sys
sys.path.insert(1, "./hdmpy")

In [105]:
# We wrap the package so that it has the familiar sklearn API
import hdmpy
from sklearn.base import BaseEstimator, clone

class RLasso(BaseEstimator):

    def __init__(self, *, post=True):
        self.post = post

    def fit(self, X, y):
        self.rlasso_ = hdmpy.rlasso(X, y, post=self.post)
        return self

    def predict(self, X):
        return np.array(X) @ np.array(self.rlasso_.est['beta']).flatten() + np.array(self.rlasso_.est['intercept'])

    def nsel(self):
        return sum(abs(np.array(self.rlasso_.est['beta']).flatten()>0))

lasso_model = lambda: RLasso(post=False)

In [106]:
# Partialling-Out using lasso

# models
X_d =  smf.ols(flex_d, data=df).data.exog[:, 1:] # exclude intercept so that lasso does not penalize it
y_d = smf.ols(flex_d, data=df).data.endog

X_y =  smf.ols(flex_y, data=df).data.exog[:, 1:] # exclude intercept so that lasso does not penalize it
y_y = smf.ols(flex_y, data=df).data.endog

# partialling-out the linear effect of W from Y
t_Y_lasso = y_y - lasso_model().fit(X_y, y_y).predict(X_y)

# partialling-out the linear effect of W from D
t_D_lasso = y_d - lasso_model().fit(X_d, y_d).predict(X_d)

# regression of Y on D after partialling-out the effect of W
partial_lasso_fit = sm.OLS(t_Y_lasso, t_D_lasso).fit()
partial_lasso_est = partial_lasso_fit.params[0]

print("Coefficient for D via partialling-out using lasso " + str(partial_lasso_est))

# standard error
partial_lasso_se = partial_lasso_fit.HC3_se[0]

# confidence interval
print("95% CI: " + str(partial_lasso_fit.conf_int()[0]))

Coefficient for D via partialling-out using lasso -0.0674754564015461
95% CI: [-0.09726648 -0.03768444]


Using lasso for partialling-out here provides similar results as using ols.

In [107]:
table2 = pd.DataFrame()

table2['Model'] = ["Without controls", "full reg", "partial reg", "partial reg via lasso"]

table2['Estimate'] = [nocontrol_est, control_est, partial_est, partial_lasso_est]

table2['Std. Error'] = [nocontrol_se, control_se, partial_se, partial_lasso_se]

# Show results
table2

,Model,Estimate,Std. Error
0,Without controls,-0.038345,0.015908
1,full reg,-0.069553,0.015699
2,partial reg,-0.069553,0.015006
3,partial reg via lasso,-0.067475,0.015441


In [108]:
# print to Latex
print(table2.style.to_latex())

\begin{tabular}{llrr}
 & Model & Estimate & Std. Error \\
0 & Without controls & -0.038345 & 0.015908 \\
1 & full reg & -0.069553 & 0.015699 \\
2 & partial reg & -0.069553 & 0.015006 \\
3 & partial reg via lasso & -0.067475 & 0.015441 \\
\end{tabular}



It it worth to notice that controlling for worker characteristics increases the wage gap from less that 4\% to 7\%. The controls we used in our analysis include 5 educational attainment indicators (less than high school graduates, high school graduates, some college, college graduate, and advanced degree), 4 region indicators (midwest, south, west, and northeast); a quartic term (first, second, third, and fourth power) in experience and 22 occupation and 23 industry indicators.

Keep in mind that the predictive effect (PE) does not only measure discrimination (causal effect of being female), it also may reflect selection effects of unobserved differences in covariates between male and female workers in our sample.

## OLS Overfitting

Next we motivate the usage of lasso. We try an "extra" flexible model, where we take interactions of all controls, giving us about 1000 controls. To illustrate how OLS can overfit, we subset to the first 1000 observations so that $p \approx n$.

In [109]:
subset,test = train_test_split(df, train_size=1000, random_state=2724)

In [110]:
# model
extraflex = "lwage ~ sex + (exp1+exp2+exp3+exp4+shs+hsg+scl+clg+C(occ2)+C(ind2)+mw+so+we)**2"

control_fit_extra = smf.ols(extraflex, data=subset).fit()
control_extra_est = control_fit_extra.params['sex']

n = subset.shape[0]
p = control_fit_extra.params.shape[0]

#Jackknife. Python regularizes in the background. Theory shouldn't really work here.
# Cattaneo, Jannson, and Newey (2018) (CJN) do verify that jackknife is valid,
# in the sense of being upward biased, in high-dimensional settings under
# regularity.
control_extra_se = control_fit_extra.HC3_se["sex"] #
# HC0 will not work here. CJN show it is inconsistent even in moderate dimensional settings.
control_extra_se_HC0 = control_fit_extra.HC0_se["sex"]
# We should probably implement Cattaneo, Jannson, and Newey (2018)'s procedure -
# though it's not clear their regularity conditions would apply here anyway.

print("Number of Extra-Flex Controls " + str( p- 1))

print("Coefficient for OLS with extra flex controls " +  str(control_extra_est))

Number of Extra-Flex Controls 979
Coefficient for OLS with extra flex controls -0.07355087637399149


In [111]:
# Partialling-Out using lasso
extraflex_y = "lwage ~  (exp1+exp2+exp3+exp4+shs+hsg+scl+clg+C(occ2)+C(ind2)+mw+so+we)**2" # model for Y
extraflex_d = "sex ~ (exp1+exp2+exp3+exp4+shs+hsg+scl+clg+C(occ2)+C(ind2)+mw+so+we)**2"   # model for D

# models
X_d =  smf.ols(extraflex_d, data=subset).data.exog[:, 1:] # exclude intercept so that lasso does not penalize it
y_d = smf.ols(extraflex_d, data=subset).data.endog

X_y =  smf.ols(extraflex_y, data=subset).data.exog[:, 1:] # exclude intercept so that lasso does not penalize it
y_y = smf.ols(extraflex_y, data=subset).data.endog

# partialling-out the linear effect of W from Y
t_Y_lasso = y_y - lasso_model().fit(X_y, y_y).predict(X_y)

# partialling-out the linear effect of W from D
t_D_lasso = y_d - lasso_model().fit(X_d, y_d).predict(X_d)

# regression of Y on D after partialling-out the effect of W
partial_lasso_fit_extra = sm.OLS(t_Y_lasso, t_D_lasso).fit()
partial_lasso_est_extra = partial_lasso_fit.params[0]

print("Coefficient for D via partialling-out using lasso " + str(partial_lasso_est_extra))

# standard error
partial_lasso_se_extra = partial_lasso_fit.HC0_se[0]


Coefficient for D via partialling-out using lasso -0.0674754564015461


In [112]:
table3 = pd.DataFrame()

table3['Model'] = ["full reg HC0", "full reg HC3", "partial reg via lasso"]

table3['Estimate'] = [control_extra_est, control_extra_est, partial_lasso_est_extra]

table3['Std. Error'] = [control_extra_se_HC0, control_extra_se, partial_lasso_se_extra]

# Show results
table3

,Model,Estimate,Std. Error
0,full reg HC0,-0.073551,0.042993
1,full reg HC3,-0.073551,0.080251
2,partial reg via lasso,-0.067475,0.015436


In [113]:
# print to Latex
print(table3.style.to_latex())

\begin{tabular}{llrr}
 & Model & Estimate & Std. Error \\
0 & full reg HC0 & -0.073551 & 0.042993 \\
1 & full reg HC3 & -0.073551 & 0.080251 \\
2 & partial reg via lasso & -0.067475 & 0.015436 \\
\end{tabular}



In this case $p/n \approx 1$, that is  $p/n$  is no longer small and we start seeing the differences between unregularized partialling out and regularized partialling out with lasso (double lasso). The results based on double lasso have rigorous guarantees in this non-small $p/n$ regime under approximate sparsity. The results based on OLS still have guarantees in $p/n< 1$ regime under assumptions laid out in Cattaneo, Newey, and Jansson (2018), without approximate sparsity, although other regularity conditions are needed.